In [1]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

2.8.0


In [2]:
x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

#convert into numpy and float format
x_data = np.asarray(x_data, dtype=np.float32)
y_data = np.asarray(y_data, dtype=np.float32)

In [3]:
nb_classes = 3 #class의 개수입니다.

print(x_data.shape)
print(y_data.shape)

(8, 4)
(8, 3)


In [4]:
#Weight and bias setting
W = tf.Variable(tf.random.normal((4, nb_classes)), name='weight')
b = tf.Variable(tf.random.normal((nb_classes,)), name='bias')
variables = [W, b]

print(W,b)

2022-05-10 23:30:03.776948: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-10 23:30:03.777445: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1
<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
array([[ 2.046651  , -0.61488664, -0.91315305],
       [ 0.66364247, -0.68873894, -0.16364992],
       [ 0.1197662 , -2.3775377 ,  0.40424803],
       [-0.30651575,  0.6726224 , -0.7714349 ]], dtype=float32)> <tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([ 0.20364496, -1.5248002 , -0.91010517], dtype=float32)>


In [80]:
def hypothesis(features):
    X = tf.matmul(features, W)
    # softmax -> tf.nn.softmax(X + b)
    # tf.reduce_sum(tf.exp(X+b), keepdims=True, axis=1) -> tf.exp(X+b)행렬의 가로축에 대해 합을 구한다 / keepdims=True -> 행렬의 차원을 유지한다
    result = tf.divide(tf.exp(X+b), tf.reduce_sum(tf.exp(X+b), keepdims=True, axis=1))
    
    return result

In [81]:
# cross-entropy
def cost_fn(x, y):
    logits = hypothesis(x)
    cost = -tf.reduce_sum(y * tf.math.log(logits))
    cost_mean = tf.reduce_mean(cost)
    
    return cost_mean

In [82]:
def grad_fn(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_fn(X, Y)
    
    grads = tape.gradient(loss, [W, b])
        
    return grads

In [83]:
def fit(X, Y, epochs=1000, verbose=100):
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)
    
    for i in range(epochs):
        optimizer.apply_gradients(zip(grad_fn(X, Y), [W, b]))
        if(i==0) | ((i+1)%verbose == 0):
             print('Loss at epoch %d: %f' %(i+1, cost_fn(X, Y).numpy()))
fit(x_data, y_data)

Loss at epoch 1: 0.007605
Loss at epoch 100: 0.007358
Loss at epoch 200: 0.007149
Loss at epoch 300: 0.006974
Loss at epoch 400: 0.006829
Loss at epoch 500: 0.006709
Loss at epoch 600: 0.006610
Loss at epoch 700: 0.006528
Loss at epoch 800: 0.006460
Loss at epoch 900: 0.006405
Loss at epoch 1000: 0.006358


In [84]:
sample_data = [[2,1,3,2]] # answer_label [[0,0,1]]
sample_data = np.asarray(sample_data, dtype=np.float32)

a = hypothesis(sample_data)

print(a)
# tf.argmax -> 배열에서 가장 큰 값을 가진 인덱스 값 반환
print(tf.argmax(a, 1)) #index: 2

tf.Tensor([[1.3868584e-10 8.1802771e-04 9.9918205e-01]], shape=(1, 3), dtype=float32)
tf.Tensor([2], shape=(1,), dtype=int64)


In [85]:
b = hypothesis(x_data)
print(b)
print(tf.argmax(b, 1))
print(tf.argmax(y_data, 1)) # matches with y_data

tf.Tensor(
[[4.4076760e-25 1.5760945e-11 1.0000000e+00]
 [1.3868594e-10 8.1802835e-04 9.9918205e-01]
 [3.8729113e-34 6.3089828e-04 9.9936914e-01]
 [2.1036039e-25 9.9910325e-01 8.9674018e-04]
 [1.6629529e-03 9.9831790e-01 1.9161424e-05]
 [4.3122098e-04 9.9956876e-01 1.7706035e-14]
 [9.9810648e-01 1.8934758e-03 2.3753940e-15]
 [9.9999815e-01 1.8728310e-06 1.3302982e-22]], shape=(8, 3), dtype=float32)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)
